Value investing means to invest in the 50 cheapest stocks that are relative to the 
common measure of business asset (earning or return)

In [1]:
import pandas as pd 
import numpy as np 
import xlsxwriter
import requests
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'QA/LOATDTGLMNEANEKCSBRS LS(G)E  A', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'ifacolif', 'close': None, 'closeTime': None, 'closeSource': 'aiffolic', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 142.28, 'latestSource': 'IEX Last Trade', 'latestTime': 'January 21, 2021', 'latestUpdate': 1667533958441, 'latestVolume': None, 'iexRealtimePrice': 137.37, 'iexRealtimeSize': 100, 'iexLastUpdated': 1660885053510, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 138.25, 'previousVolume': 107934896, 'change': 4.86, 'changePercent': 0.0372, 'volume': None, 'iexMarketPercent': 0.017054223113995758, 'iexVolume': 2044503, 'avgTotalVolume': 109159

In [4]:
data['peRatio']

42.19

## Making the Batch API call

In [5]:
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i + n]

symbol_lists = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_lists)):
    symbol_strings.append(','.join(symbol_lists[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [6]:
column_names = ['Ticker', 'Price', 'Price-to-Earning Ratio', 'Number of Shares to Buy']
df = pd.DataFrame(columns=column_names)
df

,Ticker,Price,Price-to-Earning Ratio,Number of Shares to Buy


In [7]:
for symbol_string in symbol_strings:
    batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_api).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data2[symbol]['quote']['latestPrice'],
                    data2[symbol]['quote']['peRatio'],
                    'N/A'
                ],
            index=column_names),
        ignore_index=True)
df

,Ticker,Price,Price-to-Earning Ratio,Number of Shares to Buy
0,A,131,55.27,N/A
1,AAL,16.182,-1.16,N/A
2,AAP,173.090,24.41,N/A
3,AAPL,138.330,42.98,N/A
4,ABBV,112.010,24.60,N/A
...,...,...,...,...
500,YUM,110.990,31.98,N/A
501,ZBH,168.330,-229.60,N/A
502,ZBRA,408.570,47.26,N/A
503,ZION,49.740,20.73,N/A


## Removing poor performing stocks 

In [8]:
df.sort_values('Price-to-Earning Ratio', ascending=False, inplace=True)
df = df[df['Price-to-Earning Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True, drop=True)
df.head()

,Ticker,Price,Price-to-Earning Ratio,Number of Shares to Buy
0,KMI,15.85,327.27,N/A
1,MAR,129.70,239.56,N/A
2,IQV,189.03,213.13,N/A
3,CMG,1571.40,181.67,N/A
4,REG,47.79,174.21,N/A


In [9]:
def portfolio_size():
    global capitals
    capitals = input('Please enter how much money you are going to invest: ')

    try:
        capitals = float(capitals)
    except ValueError:
        print("That's not a number.\nPlease try again.")
        capitals = input('Please enter how much money you are going to invest: ')
        capitals = float(capitals)

portfolio_size()
print(capitals)

1000000.0


In [10]:
weight = capitals / len(df.index)
weight

20000.0

In [19]:
df['Number of Shares to Buy'] = np.floor(weight / df['Price'])
df.head()

,Ticker,Price,Price-to-Earning Ratio,Number of Shares to Buy
0,KMI,15.85,327.27,1261
1,MAR,129.70,239.56,154
2,IQV,189.03,213.13,105
3,CMG,1571.40,181.67,12
4,REG,47.79,174.21,418


## More realistic model
P/E ratio
price to book ratio
price to sales ratio
EV/EBITA
EV/Gross profit

In [20]:
column_names2 = [
    'Ticker', 
    'Price', 
    'Price-to-Earning Ratio', 
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sale Ratio',
    'PS Percentile' ,
    'EV/EBITDA', 
    'EV/EBITDA Percentile',
    'EV/GP', 
    'EV/GP Percentile',
    'Number of Shares to Buy',
    ]

df2 = pd.DataFrame(columns=column_names2)
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy


In [21]:
# Detact the NoneTypes
def detactNone(a, b):
    try:
        c = a / b 
    except TypeError:
        c = None
    return c

In [22]:
for symbol_string in symbol_strings:
    batch_api2 = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data3 = requests.get(batch_api2).json()
    for symbol in symbol_string.split(','):

        price = data3[symbol]['quote']['latestPrice']
        peRatio = data3[symbol]['advanced-stats']['peRatio']
        priceToBook = data3[symbol]['advanced-stats']['priceToBook']
        priceToSales = data3[symbol]['advanced-stats']['priceToSales']
        enterpriseValue = data3[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data3[symbol]['advanced-stats']['EBITDA']
        gross_profit = data3[symbol]['advanced-stats']['grossProfit']
        evToebitda = detactNone(enterpriseValue, ebitda)
        evTogp = detactNone(enterpriseValue, gross_profit)

        df2 = df2.append(
            pd.Series(
                [
                    symbol,
                    price,
                    peRatio,
                    np.nan,
                    priceToBook,
                    np.nan,
                    priceToSales,
                    np.nan,
                    evToebitda,
                    np.nan,
                    evTogp,
                    np.nan,
                    np.nan,
                ],
            index=column_names2),
        ignore_index=True)
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy
0,A,133,56.168438,NaN,8.39,NaN,7.4100,NaN,35.188600,NaN,14.394650,NaN,NaN
1,AAL,16.408,-1.559940,NaN,-1.79,NaN,0.4135,NaN,-6.254030,NaN,1.438574,NaN,NaN
2,AAP,174.840,23.941816,NaN,3.05,NaN,1.2000,NaN,11.549056,NaN,2.583252,NaN,NaN
3,AAPL,137.760,39.839237,NaN,35.63,NaN,8.4500,NaN,29.666415,NaN,22.190157,NaN,NaN
4,ABBV,113.190,27.847914,NaN,13.58,NaN,5.0400,NaN,16.093439,NaN,9.720264,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.440,32.443584,NaN,-4.31,NaN,5.9100,NaN,23.598821,NaN,10.225671,NaN,NaN
501,ZBH,167.870,-226.763164,NaN,2.98,NaN,5,NaN,28.301254,NaN,8.261230,NaN,NaN
502,ZBRA,417.460,47.599130,NaN,11.49,NaN,5,NaN,33.565183,NaN,12.251766,NaN,NaN
503,ZION,48.800,20.685786,NaN,1.15,NaN,2.6900,NaN,11.684551,NaN,2.953018,NaN,NaN


In [23]:
lookup = [
    'Price-to-Earning Ratio', 
    'Price-to-Book Ratio',
    'Price-to-Sale Ratio',
    'EV/EBITDA', 
    'EV/GP',
]

for i in lookup:
    na_df = df2[df2[i].isnull()]

len(na_df.index)


17

## Droping the rows that have at least 7 NA

In [24]:
df2.dropna(thresh=7, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy
0,A,133,56.168438,NaN,8.39,NaN,7.4100,NaN,35.188600,NaN,14.394650,NaN,NaN
1,AAL,16.408,-1.559940,NaN,-1.79,NaN,0.4135,NaN,-6.254030,NaN,1.438574,NaN,NaN
2,AAP,174.840,23.941816,NaN,3.05,NaN,1.2000,NaN,11.549056,NaN,2.583252,NaN,NaN
3,AAPL,137.760,39.839237,NaN,35.63,NaN,8.4500,NaN,29.666415,NaN,22.190157,NaN,NaN
4,ABBV,113.190,27.847914,NaN,13.58,NaN,5.0400,NaN,16.093439,NaN,9.720264,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,YUM,108.440,32.443584,NaN,-4.31,NaN,5.9100,NaN,23.598821,NaN,10.225671,NaN,NaN
484,ZBH,167.870,-226.763164,NaN,2.98,NaN,5,NaN,28.301254,NaN,8.261230,NaN,NaN
485,ZBRA,417.460,47.599130,NaN,11.49,NaN,5,NaN,33.565183,NaN,12.251766,NaN,NaN
486,ZION,48.800,20.685786,NaN,1.15,NaN,2.6900,NaN,11.684551,NaN,2.953018,NaN,NaN


## Calculating the percentiles

In [25]:
metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sale Ratio': 'PS Percentile' ,
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
}
metrics.keys()

dict_keys(['Price-to-Earning Ratio', 'Price-to-Book Ratio', 'Price-to-Sale Ratio', 'EV/EBITDA', 'EV/GP'])

In [36]:
df2['RV Score'] = np.nan
for row in df2.index:
    for metric in metrics:
        df2.loc[row, metrics[metric]] = stats.percentileofscore(df2[metric], df2.loc[row, metric])
    df2.loc[row, 'RV Score'] = np.mean(df2[metrics.values()].iloc[row])
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy,RV Score
0,A,133,56.168438,86.475410,8.39,77.049180,7.4100,80.532787,35.188600,88.319672,14.394650,81.557377,NaN,82.786885
1,AAL,16.408,-1.559940,15.573770,-1.79,4.918033,0.4135,2.868852,-6.254030,3.073770,1.438574,5.532787,NaN,6.393443
2,AAP,174.840,23.941816,47.131148,3.05,42.213115,1.2000,17.930328,11.549056,28.278689,2.583252,12.295082,NaN,29.569672
3,AAPL,137.760,39.839237,76.024590,35.63,96.311475,8.4500,84.426230,29.666415,84.426230,22.190157,95.081967,NaN,87.254098
4,ABBV,113.190,27.847914,55.327869,13.58,86.270492,5.0400,69.057377,16.093439,48.360656,9.720264,64.344262,NaN,64.672131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,YUM,108.440,32.443584,64.549180,-4.31,4.508197,5.9100,73.155738,23.598821,73.565574,10.225671,65.573770,NaN,56.270492
484,ZBH,167.870,-226.763164,2.663934,2.98,40.778689,5,68.032787,28.301254,82.377049,8.261230,53.893443,NaN,49.549180
485,ZBRA,417.460,47.599130,81.147541,11.49,82.991803,5,68.032787,33.565183,87.500000,12.251766,73.770492,NaN,78.688525
486,ZION,48.800,20.685786,41.393443,1.15,11.475410,2.6900,43.340164,11.684551,29.303279,2.953018,14.549180,NaN,28.012295


In [44]:
df2.sort_values('RV Score', ascending=True, inplace=True)
df2 = df2[:50]
df2.reset_index(drop=True, inplace=True)
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy,RV Score
0,UNM,26.380,5.799177,17.418033,0.5070,5.532787,0.4625,3.483607,2.902766,3.688525,0.437974,0.409836,388,6.106557
1,AIG,41.560,-7.289359,11.680328,0.5877,5.942623,0.8299,7.991803,4.832551,4.508197,0.809147,1.639344,246,6.352459
2,AAL,16.408,-1.559940,15.573770,-1.7900,4.918033,0.4135,2.868852,-6.254030,3.073770,1.438574,5.532787,624,6.393443
3,F,12.080,-265.453723,2.459016,1.3200,15.471311,0.3265,1.229508,4.416142,4.098361,2.186861,10.450820,848,6.741803
4,MET,52.130,8.551159,19.057377,0.6560,6.352459,0.7276,6.352459,4.891972,4.713115,0.712548,1.229508,196,7.540984
5,ALL,114.360,7.728059,18.032787,1.3500,16.393443,0.7582,6.762295,2.523161,3.483607,0.735446,1.434426,89,9.221311
6,FTI,11.440,-0.940005,16.188525,1.2600,13.729508,0.3850,2.254098,3.515135,3.893443,2.259249,11.065574,895,9.426230
7,BA,212.310,-27.138734,7.786885,-10.4000,3.483607,2.0500,34.733607,-29.285999,1.639344,-204.248854,0.204918,48,9.569672
8,HIG,51.050,10.989657,20.696721,1.1100,10.758197,0.9468,10.245902,4.469974,4.303279,0.896334,2.254098,200,9.651639
9,ABC,105.020,-6.334395,12.295082,-21.0700,2.049180,0.1155,0.409836,8.514184,12.704918,4.072457,23.155738,97,10.122951


In [38]:
portfolio_size()
print(capitals)

5000000.0


In [45]:
equal_weight = float(capitals / len(df2.index))
df2['Number of Shares to Buy'] = np.floor(equal_weight / df2['Price'])
df2

,Ticker,Price,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Number of Shares to Buy,RV Score
0,UNM,26.380,5.799177,17.418033,0.5070,5.532787,0.4625,3.483607,2.902766,3.688525,0.437974,0.409836,3790,6.106557
1,AIG,41.560,-7.289359,11.680328,0.5877,5.942623,0.8299,7.991803,4.832551,4.508197,0.809147,1.639344,2406,6.352459
2,AAL,16.408,-1.559940,15.573770,-1.7900,4.918033,0.4135,2.868852,-6.254030,3.073770,1.438574,5.532787,6094,6.393443
3,F,12.080,-265.453723,2.459016,1.3200,15.471311,0.3265,1.229508,4.416142,4.098361,2.186861,10.450820,8278,6.741803
4,MET,52.130,8.551159,19.057377,0.6560,6.352459,0.7276,6.352459,4.891972,4.713115,0.712548,1.229508,1918,7.540984
5,ALL,114.360,7.728059,18.032787,1.3500,16.393443,0.7582,6.762295,2.523161,3.483607,0.735446,1.434426,874,9.221311
6,FTI,11.440,-0.940005,16.188525,1.2600,13.729508,0.3850,2.254098,3.515135,3.893443,2.259249,11.065574,8741,9.426230
7,BA,212.310,-27.138734,7.786885,-10.4000,3.483607,2.0500,34.733607,-29.285999,1.639344,-204.248854,0.204918,471,9.569672
8,HIG,51.050,10.989657,20.696721,1.1100,10.758197,0.9468,10.245902,4.469974,4.303279,0.896334,2.254098,1958,9.651639
9,ABC,105.020,-6.334395,12.295082,-21.0700,2.049180,0.1155,0.409836,8.514184,12.704918,4.072457,23.155738,952,10.122951


In [46]:
df2.to_excel('Value_investing.xlsx')